In [ ]:
from openai import OpenAI
import base64
import os

In [ ]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
image_path = "shuaidao.png"
base64_image = encode_image(image_path)
data_url = 'data:image/png;base64,'+base64_image

In [ ]:
client = OpenAI(
    base_url='https://api.siliconflow.cn/v1',
    api_key='sk-pawqcnvkrpbjokrapqcqfjnwcpfastyomscrzcscnbbqwywi'
)

response = client.chat.completions.create(
    model="Pro/Qwen/Qwen2.5-VL-7B-Instruct",
    messages=[
        {"role": "user", 
         "content": [
             {'type': 'text',
              'text': '请描述一下这张图片内容，我作为一个社区机器人，我应该做什么'},
             {'type': 'image_url',
              'image_url': {
                  'url': data_url,
                  'detail': 'auto'
              }}
         ]}
    ],
    stream=False  # 启用流式输出
)

In [ ]:
print(response.choices[0].message.content)

In [ ]:
import requests
import json

def detect_objects(image_path, api_url="http://localhost:8000/detect/"):
    """
    调用YOLO11检测API
    
    参数:
        image_path: 图像文件路径
        api_url: API地址
        
    返回:
        API返回的检测结果
    """
    # 准备文件
    with open(image_path, "rb") as image_file:
        files = {"file": (image_path, image_file, "image/jpeg")}
        
        # 发送POST请求到API
        response = requests.post(api_url, files=files)
        
    # 检查响应
    if response.status_code == 200:
        # 解析JSON响应
        results = response.json()
        print(results)
        print(f"检测到 {results['total']} 个对象:")
        
        for det in results["detections"]:
            print(f"- 类别: {det['class_name']}, 置信度: {det['confidence']:.2f}")
            
        return results
    else:
        print(f"错误: {response.status_code}, {response.text}")
        return None

In [ ]:
if __name__ == "__main__":
    # 替换为你的图像路径
    image_path = "shuaidao.jpg"
    
    # 替换为你的API地址（如果在远程服务器上）
    # api_url = "http://your_server_ip:8007/detect/"
    api_url = "http://localhost:8007/detect/"
    
    results = detect_objects(image_path, api_url)

In [ ]:
from typing import Literal